Welcome to assignment 1.                                                       

We are using pathology images for our first assignment please download data from this link https://drive.google.com/drive/folders/10dUOzcPR-PQwfFYcHk5gsLjIjSorQ32Q?usp=sharing



Task 1: Feature Generation (15%)
Use and run the following code (a deep network) to generate features from a set of training images. For this assignment, you do not need to know how the deep network is working here to extract features.
This code extracts the features of image T4.tif (in the T folder of dataset). Modify the code so that it iterates over all images of the dataset and extracts their features.
Allocate 10% of the data for validation.

Insert your code here for Task 1





In [16]:
# import the necessary packages

import torch
import numpy as np
import torchvision.transforms as transforms
from torchvision.models import densenet121
from torch.autograd import Variable
from PIL import Image
from sklearn.cluster import DBSCAN
from sklearn.metrics import silhouette_score
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
import os

In [17]:
# Load pre-trained DenseNet model
model = densenet121(pretrained=True)

# Remove the classification layer (last fully connected layer)
model = torch.nn.Sequential(*list(model.children())[:-1])

# Add a global average pooling layer
model.add_module("global_avg_pool", torch.nn.AdaptiveAvgPool2d(1))

# Set the model to evaluation mode

model.eval()

# Define the image preprocessing pipeline

preprocess = transforms.Compose(
    [
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ]
)

# # Load an image
# image_path = "train\T\T4.tif"
# image = Image.open(image_path)
# Preprocess the image
input_tensor = preprocess(image)

input_batch = input_tensor.unsqueeze(0)  # Add a batch dimension

# Wrap the input tensor in a Variable

input_var = Variable(input_batch)

# Forward pass through the model

features = model(input_var)

# Extract the feature tensor

feature_vector = features.squeeze().detach().numpy()

# Now 'feature_vector' contains the feature from the last fully connected layer of DenseNet

print("Feature vector shape:", feature_vector.shape)

# Directory containing the data folder

data_dir = "train"

# List all directories

directories = [
    folder
    for folder in os.listdir(data_dir)
    if os.path.isdir(os.path.join(data_dir, folder))
]

# Initialize lists for storing images and labels

images = []
labels = []
features = []

for directory in directories:
    folder_path = os.path.join(data_dir, directory)
    if os.path.isdir(folder_path):
        # List all files in the directory
        files = os.listdir(folder_path)
        # Filter out .tif files
        tif_files = [file for file in files if file.endswith(".tif")]
        # Load each .tif file
        for file in tif_files:
            image, feature_vector = extract_features(os.path.join(folder_path, file))
            images.append(image)

            # Append the label (folder name) to the list
            labels.append(directory)
            # Append feature vector
            features.append(feature_vector)


# Split paths and labels into train and validation sets

train_features, val_features, train_labels, val_labels = train_test_split(
    features, labels, test_size=0.1, random_state=42
)

Feature vector shape: (1024,)


TypeError: extract_features() missing 2 required positional arguments: 'model' and 'preprocess'

 Task 2: High Bias Classification Method (5%)
 Choose a classification method and let is have a high bias.
 Train it on the generated features and discuss why it is underfitting.

 Insert your code here for Task 2




In [19]:
# Use a multi-class logistic regression method to classify data
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.model_selection import cross_val_score

# initialize a logistic regression model
lr_model = LogisticRegression(max_iter=1000, multi_class="ovr")

# perform k-fold cross validation
k = 10
lr_scores = cross_val_score(lr_model, train_features, train_labels, cv=k)

# Print cross-validation scores for logistic regression
for i, score in enumerate(lr_scores):
    print(f"Logistic Regression Fold {i+1} Accuracy: {score}")

# Print mean scores for logistic regression
mean_lr_accuracy = np.mean(lr_scores)
print(f"Mean Logistic Regression Accuracy: {mean_lr_accuracy}")

# Use a multi-class SVM method to classify data
hb_svm_model = svm.SVC(kernel="linear", C=0.01, gamma=1000)
hb_svm_scores = cross_val_score(hb_svm_model, train_features, train_labels, cv=k)

# Print the cross-validation scores
for i, hb_score in enumerate(hb_svm_scores):
    print(f"SVM Fold {i+1} Accuracy: {hb_score}")

# Calculate and print the mean accuracy across all folds
hb_mean_accuracy = np.mean(hb_svm_scores)
print(f"Mean SVM Accuracy: {hb_mean_accuracy}")

# Calculate and print the accuracy variance
hb_accuracy_var = np.var(hb_svm_scores)  # unterminated string literal
print(f"SVM Accuracy Variance: {hb_accuracy_var}")

NameError: name 'train_features' is not defined

 Task 3: High Variance Classification Method (5%)
 Use the chosen classification method and let it have a high variance.
 Train it on the generated features and discuss why it is overfitting.

 Insert your code here for Task 3




In [ ]:
# Use a multi-class SVM method to classify data
hv_svm_model = svm.SVC(kernel='sigmoid')
hv_svm_scores = cross_val_score(hv_svm_model, train_features, train_labels, cv=k)

# Print the cross-validation scores
for i, hv_score in enumerate(hv_svm_scores):
    print(f"SVM Fold {i+1} Accuracy: {hv_score}")

# Calculate and print the mean accuracy across all folds
hv_mean_accuracy = np.mean(hv_svm_scores)
print(f"Mean SVM Accuracy: {hv_mean_accuracy}")

# Calculate and print the accuracy variance across all folds
hv_accuracy_var = np.var(hv_svm_scores)
print(f"SVM Accuracy Variance: {hv_accuracy_var}")

 Task 4: Balanced Classification Method (15%)
 Use the chosen classification method and let it balance the bias and variance.
 Train it on the generated features, possibly adjusting parameters.
 Discuss insights into achieving balance.

 Insert your code here for Task 4




In [ ]:
balanced_svm_model = svm.SVC(kernel='linear')
balanced_svm_scores = cross_val_score(balanced_svm_model, train_features, 
                                      train_labels, cv=k)

# Print the cross-validation scores
for i, balanced_score in enumerate(balanced_svm_scores):
    print(f"SVM Fold {i+1} Accuracy: {balanced_score}")

# Calculate and print the mean accuracy across all folds
balanced_mean_accuracy = np.mean(balanced_svm_scores)
print(f"Balanced SVM Mean Accuracy: {balanced_mean_accuracy}")

# Calculate and print the accuracy variance across all folds
balanced_accuracy_var = np.var(balanced_svm_scores)
print(f"Balanced SVM Accuracy Variance: {balanced_accuracy_var}")

 Task 5: K-Means Clustering (20%)
 Apply K-Means clustering on the generated features.
 Test with available labels and report accuracy.
 Experiment with automated K and compare with manually set 20 clusters.

 Insert your code here for Task 5




In [18]:

# Assuming 'train_features' are the features extracted from the images

# Apply K-Means with manually set 20 clusters
kmeans_20 = KMeans(n_clusters=20, random_state=42)
clusters_20 = kmeans_20.fit_predict(train_features)

# Since KMeans does not inherently provide labels matching to original labels,
# a mapping function or strategy is needed to evaluate clustering accuracy.

# Apply K-Means with automated K (e.g., using the Elbow method or other heuristic)
# This step would involve determining the optimal number of clusters K,
# which can be done using methods like the Elbow Method or the Silhouette Score.

# Example for Elbow Method (commented out because it requires plotting)


# wcss = []
# for i in range(1, 21):
#     kmeans = KMeans(n_clusters=i, random_state=42)
#     kmeans.fit(train_features)
#     wcss.append(kmeans.inertia_)
# plt.plot(range(1, 21), wcss)
# plt.title('Elbow Method')
# plt.xlabel('Number of clusters')
# plt.ylabel('WCSS')  # Within cluster sum of squares
# plt.show()

# Assuming optimal K is found, e.g., k_optimal
# kmeans_opt = KMeans(n_clusters=k_optimal, random_state=42)
# clusters_opt = kmeans_opt.fit_predict(train_features)

# Accuracy reporting would require a strategy to match cluster labels to true labels, which is non-trivial
# because clustering is unsupervised and does not directly produce class labels that match with the ground truth.

NameError: name 'train_features' is not defined

 Task 6: Additional Clustering Algorithm (10%)
 Choose another clustering algorithm and apply it on the features.
 Test accuracy with available labels.

 Insert your code here for Task 6




In [ ]:


# Assuming 'train_features' are the features extracted from the images

# Apply DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_clusters = dbscan.fit_predict(train_features)

# The silhouette score can be used to evaluate the quality of the clustering
silhouette_avg = silhouette_score(train_features, dbscan_clusters)
print(f"Silhouette Score: {silhouette_avg}")

# DBSCAN labels outliers as -1, so you might want to handle them in your accuracy assessment


 Task 7: PCA for Classification Improvement (20%)
 Apply PCA on the features and then feed them to the best classification method in the above tasks.
 Assess if PCA improves outcomes and discuss the results.

 Insert your code here for Task 7




 Task 8: Visualization and Analysis (10%)
 Plot the features in a lower dimension using dimentinality reduction techniques.
 Analyze the visual representation, identifying patterns or insights.

Insert your code here for Task 8